In [1]:
# try:
#     get_ipython().system('pip install pandas_datareader')
#     get_ipython().system('pip install pandas')
#     get_ipython().system('pip install matplotlib')
#     get_ipython().system('pip install requests')
#     get_ipython().system('pip install sympy')
#     get_ipython().system('pip install scipy')
#     get_ipython().system('pip install numpy')
#     get_ipython().system('pip install beautifulsoup4')
#     get_ipython().system('pip install numpy-financial')
# except Exception:
#     pass

In [2]:
import requests
import numpy_financial as npf
import numpy as np
from bs4 import BeautifulSoup
import threading
import re
import math

In [3]:
tickers_proceed = []
tickers = []

# with open('analysis/nasdaq.csv') as fh:
#     for line in fh:
#         ticker = line.strip().split('|')[0]
#         tickers_proceed.append(ticker)

with open('tickers.csv') as fh:
    for line in fh:
        ticker = line.strip()
        if ticker not in tickers_proceed:
            tickers.append(ticker)


In [4]:
# jitta temp mail: fesodiha@clout.wiki
# copy this following code to extract cookies from jitta.com and then manual replace number to string for some key, remove nested field
# document.cookie.split(';').map((c) => c.trim().split('=').map(decodeURIComponent)).reduce((a, b) => {try {a[b[0]] = JSON.parse(b[1]);}catch(e) {a[b[0]] = b[1];}return a;}, {});

cookies = {
    "_gid": "GA1.2.1391914654.1680622374",
    "_gat_UA-135483057-1": "1",
    "_gat_UA-148344773-2": "1",
    "gclid": "undefined",
    "ajs_anonymous_id": "bfaa853a-095e-4fe2-8e45-a4744978bd86",
    "_fbp": "fb.1.1680622377036.1559926736",
    "hunt7586-_zldp": "ss+YrC93NjPHqz8ywkj26duUYHukZxd+LtXqi65CJTRkihvOvRSMuYB9EJFHw2//7uRfjfIQJjw=",
    "hunt7586-_zldt": "12c1de6a-1ae9-45bc-9f58-3fc4d8d62c92-0",
    "_gat_UA-35429274-1": "1",
    "JDCID": "e46d6b3260a2148bf7f81fe2a37b3e80e1308da58ea37bf86426943e457c45ca.5439c914472955e6ecc4ac72cbf4f297dafa500b94dc38a5e2b63278cc6c5369275321157f5bbd49210a4b2cc4dec3851703061b2613e31a45162e61f94510a263b434e1405ea20bb920a0f8f433b0620c295848890c",
    "_ga": "GA1.1.1445677770.1680622374",
    "_ga_BT0RBD348J": "GS1.1.1680622374.1.1.1680622395.39.0.0",
    "ajs_user_id": "642bec1b2e042000174a3d46",
    "jitta_nu": "2"
}

In [5]:
def convert_row_to_data(row):
    r = row.find_all('span')
    c = map(lambda x: p2f(x.text), r)
    return list(c)[:-2]

def p2f(x):
    cx = x.replace('%', '').replace(',', '').replace('M', '')
    return None if cx == '- -' or cx == '?' else float(cx)

def rate_of_change(nums):
    return list(map(lambda i: None if nums[i-1] is None or nums[i] is None else nums[i] / nums[i-1] - 1, range(1, len(nums))))

def mean(nums):
    nums = list(filter(lambda x: x is not None, nums))
    return 0.0 if len(nums) == 0 else sum(nums) / len(nums)

def median(nums):
    nums = sorted(filter(lambda x: x is not None, nums))
    n = len(nums)
    if n == 0:
        return 0.0
    if n % 2 == 1:
        return nums[n // 2]
    else:
        return (nums[n // 2 - 1] + nums[n // 2]) / 2


def clean_data(y):
    n = len(y)
    if n == 0:
        return []
    cleaned_y = [0.0] * n
    cleaned_y[0] = y[0] if y[0] is not None else 0.0
    cleaned_y[-1] = y[-1] if y[-1] is not None else 0.0
    for i in range(1, n-1):
        if y[i] is None:
            cleaned_y[i] = 0.0
        elif abs(y[i] - cleaned_y[i-1]) > abs(y[i] - y[i+1]):
            cleaned_y[i] = y[i+1]
        else:
            cleaned_y[i] = y[i-1]
    return cleaned_y


def linear_regression(y):
    cleaned_y = clean_data(y)
    n = len(cleaned_y)
    x = list(range(n))
    x_mean = sum(x) / n
    y_mean = sum(cleaned_y) / n
    cov = sum([(x[i] - x_mean) * (cleaned_y[i] - y_mean) for i in range(n)])
    var = sum([(x[i] - x_mean) ** 2 for i in range(n)])
    slope = cov / var
    intercept = y_mean - slope * x_mean
    y_pred = [slope * i + intercept for i in x]
    roc = 0 if y_mean == 0 else slope / y_mean
    return x, slope, intercept, y_pred, roc, y_mean

def dcf_irr(price, eps, g, dpr):
    tax_rate = 0.15
    adj_dpr = (1 + dpr * (1 - tax_rate))
    cashflows = [price * -1]
    g1 = g
    g2 = 0.5 * g
    g3 = min(0.10, 0.2 * g)
    g4 = min(0.04, 0.1 * g)
    last_eps = eps
    
    for i in range(0, 40):
        if i < 5:
            last_eps = last_eps * (1 + g1)
        elif i < 10:
            last_eps = last_eps * (1 + g2)
        elif i < 20:
            last_eps = last_eps * (1 + g3)
        else:
            last_eps = last_eps * (1 + g4)

        cashflows.append(last_eps)
    irr = npf.irr(cashflows)
    return 0.0 if np.isnan(irr) else irr

In [6]:
def calculate(ticker):
    with requests.post('https://www.jitta.com/stock/' + ticker + '/factsheet', cookies=cookies) as r:
        soup = BeautifulSoup(r.content, 'lxml')

        try:
            price = float(re.search(
                r"\.price\.latest\":{\"datetime\":\"(.*)\",\"close\":(.*),\"market_cap\"", r.content.decode('utf-8'))[2])
            
            row = soup.find_all('div', {'class': 'dAGwCX'})

            if len(row) == 0:
                return None
        except:
            return None

        REV = convert_row_to_data(row[0])
        EPS = convert_row_to_data(row[1])
        ROE = convert_row_to_data(row[20])
        ROA = convert_row_to_data(row[21])
        DE = convert_row_to_data(row[23])
        DPR = convert_row_to_data(row[24])

        eps_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else x, EPS)))
        dpr_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else max(0, min(100, x)), DPR)))

        eps = float(eps_forecast[3][-1])
        eps_pos = len(list(filter(lambda x: x is not None and x > 0, EPS)))
        g = float(eps_forecast[4])
        dpr = float(dpr_forecast[3][-1] / 100)

        irr = float(dcf_irr(price, eps, g, dpr))

        print(f"{ticker} {irr:.2} {g:.2} {eps:.2} {dpr:.2}")

        return {
            "TICKER": ticker,
            "PRICE": price,
            "EPS": eps,
            "EPS_POS": eps_pos,
            "EPS_N": len(EPS),
            "G": g,
            "DPR": dpr,
            "IRR": irr,
        }


In [7]:
FILE_PREFIX = 'set'

def run_fetch_stock(tickers, thread_no):
  f = open(f"data/{FILE_PREFIX}-{thread_no}.csv", 'w')
  f.write('ticker|IRR|G|EPS|EPS_POS|EPS_N|DPR|PRICE\n')
  f.flush()
  for ticker in tickers:
    # print(ticker)
    fetch_stock(ticker, f)
    
  f.close()
  

def fetch_stock(ticker, f):
    data = calculate(ticker)

    if data == None:
        return

    f.write(ticker + '|' +
            '{:.4f}'.format(data['IRR']) + '|' +
            '{:.4f}'.format(data['G']) + '|' +
            '{:.4f}'.format(data['EPS']) + '|' +
            '{:.4f}'.format(data['EPS_POS']) + '|' +
            '{:.4f}'.format(data['EPS_N']) + '|' +
            '{:.4f}'.format(data['DPR']) + '|' +
            '{:.4f}'.format(data['PRICE']) + '\n'
            )
    f.flush()
    


In [8]:
threads  = []
total_thread = 10
split_tickers = np.array_split(tickers, total_thread)

for i in range(0, total_thread):
    thread = threading.Thread(target=run_fetch_stock, args=(split_tickers[i], i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


nyse:ECVT 4.0 -1.3e+01 0.87 0.0
nyse:TROX 1.1e+01 8.2 4.5 0.11
nyse:PD 0.0 0.36 -1.1 0.0
nyse:A 0.11 0.2 4.1 0.22
nyse:SBR -0.041 -0.12 1.4 0.63
nyse:BAK 0.12 -3.1 -1.0 -0.079
nyse:GGT 0.024 0.0042 0.28 -0.029
nyse:JHX 0.077 0.086 0.74 0.37
nyse:CINT 0.21 0.38 0.18 0.0
nyse:MSA 0.071 0.089 4.0 0.45
nyse:AA 0.0 -0.027 -1.4 0.0
nyse:ED 0.066 0.00082 4.1 0.77
nyse:TRP 0.0 -0.26 -0.0091 -0.15
nyse:JILL 1.4e+01 -4e+01 -1.1e+01 0.0
nyse:BALL 0.088 0.096 2.1 0.38
nyse:MSB 0.43 0.17 4.0 0.77
nyse:SBS 0.09 -0.01 0.8 0.28
nyse:CIO -0.17 -0.38 0.091 0.19
nyse:TRTL 0.21 0.6 0.12 0.0
nyse:PDM 0.064 -0.0011 0.79 0.074
nyse:AAC 0.18 0.53 0.12 0.0
nyse:EDD -0.25 -0.68 0.32 0.044
nyse:JKS -0.15 -0.23 0.15 0.0
nyse:SBSW 0.59 0.33 1.9 0.087
nyse:MSC 0.0 -0.029 -1.6 0.0
nyse:BALY 0.0 8.8 -2.0 0.05
nyse:GHC 0.041 -0.051 3.7e+01 0.24
nyse:TRTN 0.32 0.18 7.5 0.43
nyse:CION -0.041 -0.15 0.31 0.24
nyse:AAIC 0.0 -0.16 -0.33 0.2
nyse:JLL 0.14 0.078 1.3e+01 0.01
nyse:MSCI 0.1 0.22 1e+01 0.45
nyse:GHG 0.052 -0.034